<a href="https://colab.research.google.com/github/Abi01Er/Abi01Er/blob/main/DP%20and%20Federated%20learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install syft==0.2.9

Requested syft==0.2.9 from https://files.pythonhosted.org/packages/1c/73/891ba1dca7e0ba77be211c36688f083184d8c9d5901b8cd59cbf867052f3/syft-0.2.9-py3-none-any.whl has invalid metadata: Expected matching RIGHT_PARENTHESIS for LEFT_PARENTHESIS, after version specifier
    tf-encrypted (<0.6.0!=0.5.7,>=0.5.4) ; extra == 'udacity'
                 ~~~~~~~^
Please use pip<24.1 if you need to use this version.
ERROR: Could not find a version that satisfies the requirement syft==0.2.9 (from versions: 0.1.0a1, 0.1.1a2, 0.1.2a1, 0.1.3a1, 0.1.4a1, 0.1.4a2, 0.1.5a1, 0.1.6a1, 0.1.7a1, 0.1.8a1, 0.1.9a1, 0.1.10a1, 0.1.10a2, 0.1.10a4, 0.1.11a1, 0.1.12a1, 0.1.13a1, 0.1.14a1, 0.1.15a1, 0.1.16a1, 0.1.19a1, 0.1.20a1, 0.1.21a1, 0.1.22a1, 0.1.23a1, 0.1.24a1, 0.1.25a1, 0.1.26a1, 0.1.27a1, 0.1.28a1, 0.1.29a1, 0.2.0a1, 0.2.0a2, 0.2.1a1, 0.2.2a1, 0.2.3a1, 0.2.3a2, 0.2.3a3, 0.2.3, 0.2.4, 0.2.5, 0.2.6, 0.2.7, 0.2.8, 0.2.9, 0.3.0, 0.5.0rc1, 0.5.0rc2, 0.5.0rc3, 0.5.0, 0.5.1, 0.6.0a0, 0.6.0a1, 0.6.0a2, 0.6.0a3, 0.6.

In [ ]:
# import our libraries
import numpy as np
import pandas as pd
import torch
from torchvision import datasets, transforms,models
from torch.utils.data import Dataset, Subset, DataLoader
from torch import nn, optim
import torch.nn.functional as F
from PIL import Image
import time, os, random


# libary from pysyft needed to perform pate analysis
# from syft.frameworks.torch.dp import pate

# we'll train on GPU if it is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
class CovidCTDataset(Dataset):
    def __init__(self, root_dir, txt_COVID, txt_NonCOVID, transform=None):
        """
        Args:
            txt_path (string): Path to the txt file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        File structure:
        - root_dir
            - CT_COVID
                - img1.png
                - img2.png
                - ......
            - CT_NonCOVID
                - img1.png
                - img2.png
                - ......
        """
        self.root_dir = root_dir
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.num_cls = len(self.classes)
        self.img_list = []
        for c in range(self.num_cls):
            cls_list = [[os.path.join(self.root_dir,self.classes[c],item), c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list
        self.transform = transform


    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.img_list[idx][0]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        label = int(self.img_list[idx][1])
        return image, label

def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data


In [ ]:
batchsize=16
path = '/content/drive/MyDrive/data/images'

# Transforms used for datasets
data_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomResizedCrop((224),scale=(0.5,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# divided among teachers
trainset = CovidCTDataset(root_dir=f'{path}',
                              txt_COVID='/content/drive/MyDrive/data/labels/COVID/trainCT_COVID.txt',
                              txt_NonCOVID='/content/drive/MyDrive/data/labels/NonCOVID/trainCT_NonCOVID.txt',
                              transform= data_transforms)

# used as student valid set
validset = CovidCTDataset(root_dir=f'{path}',
                              txt_COVID='/content/drive/MyDrive/data/labels/COVID/valCT_COVID.txt',
                              txt_NonCOVID='/content/drive/MyDrive/data/labels/NonCOVID/valCT_NonCOVID.txt',
                              transform= data_transforms)

# used as student train set
testset = CovidCTDataset(root_dir=f'{path}',
                              txt_COVID='/content/drive/MyDrive/data/labels/COVID/testCT_COVID.txt',
                              txt_NonCOVID='/content/drive/MyDrive/data/labels/NonCOVID/testCT_NonCOVID.txt',
                              transform= data_transforms)

print("Number of Classes: ",len(trainset.classes))
len(trainset), len(testset), len(validset)

In [ ]:
data_loader = DataLoader(trainset, batch_size=batchsize, shuffle=True)

import matplotlib.pyplot as plt

## Method to display Image for Tensor
def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    #print(type(image))
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    return ax


# Displaying Images and other info about the train set
images, labels = next(iter(data_loader))
print(" Image Size",images.size())
# print(" Image Size",images[ii].size())

fig, axes = plt.subplots(figsize=(16,5), ncols=5)
for ii in range(5):
    ax = axes[ii]
    ax.set_title(labels[ii])
    print(" Image Size",images[ii].size())
    imshow(images[ii], ax=ax, normalize=True)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/images/CT_COVID/2020.02.13.20022673-p13-77%2.png'

In [ ]:
# TEACHERS
#divide train set among teachers and create dataloaders for valid and trainsets
num_teachers = 5
valid_per = 0.2 #20% for validation
batch_size = 32

def teacher_dataloaders(transet=trainset, num_teachers=num_teachers, batch_size=batch_size, valid_per = 0.2):
  trainloaders = []
  validloaders = []
  teacher_data_len = len(trainset) // num_teachers

  # create a list of shuffled indices
  my_list = random.sample(range(1,len(trainset)), len(trainset)-1)
  random.shuffle(my_list)

  for i in range(num_teachers):
    # get particular subset of data
    indice = my_list[i*teacher_data_len: (i+1)*teacher_data_len]
    data_subset = Subset(trainset, indice)

    # split into train and validation set
    valid_size = int(len(data_subset) * valid_per)
    train_size = len(data_subset) - valid_size
    train_subset, valid_subset = torch.utils.data.random_split(data_subset, [train_size,valid_size])

    #create data loaders
    trainloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=1)
    validloader = DataLoader(valid_subset, batch_size=batch_size, shuffle=False, num_workers=1)

    #add dataloaders to list
    trainloaders.append(trainloader)
    validloaders.append(validloader)

  return trainloaders, validloaders

# creating dataloaders
trainloaders, validloaders = teacher_dataloaders()
len(trainloaders), len(validloaders)

(5, 5)

In [ ]:
#  # STUDENT
# split into train and validation set
valid_size = int(len(testset) * 0.2)
train_size = len(testset) - valid_size
student_train_subset, student_valid_subset = torch.utils.data.random_split(testset, [train_size,valid_size])

#create data loaders
student_train_loader = DataLoader(student_train_subset, batch_size=batch_size, shuffle=False, num_workers=1)
student_valid_loader = DataLoader(student_valid_subset, batch_size=batch_size, shuffle=False, num_workers=1)

len(student_train_loader), len(student_valid_loader)

(6, 2)

#Training labelled datasets-Teachers

In [ ]:
class SimpleCNN(torch.nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__() # b, 3, 32, 32
        layer1 = torch.nn.Sequential()
        layer1.add_module('conv1', torch.nn.Conv2d(3, 32, 3, 1, padding=1))

        #b, 32, 32, 32
        layer1.add_module('relu1', torch.nn.ReLU(True))
        layer1.add_module('pool1', torch.nn.MaxPool2d(2, 2))
        self.layer1 = layer1
        layer4 = torch.nn.Sequential()
        layer4.add_module('fc1', torch.nn.Linear(401408, 2))
        self.layer4 = layer4

    def forward(self, x):
        conv1 = self.layer1(x)
        fc_input = conv1.view(conv1.size(0), -1)
        fc_out = self.layer4(fc_input)

        return fc_out

In [ ]:
def train(n_epochs, trainloader, validloader, model, optimizer, criterion, save_path=None, is_not_teacher=False):
    """returns trained model"""
    # Initialize tracker for minimum validation loss
    valid_loss_min = np.Inf

    for epoch in range(1, n_epochs + 1):
        # Initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        train_correct = 0.0
        train_total = 0.0
        valid_correct = 0.0
        valid_total = 0.0

        # Train the model
        model.train()
        for batch_idx, (data, target) in enumerate(trainloader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * data.size(0)
            _, pred = torch.max(output, 1)
            train_correct += torch.sum(pred == target.data)
            train_total += data.size(0)

        train_loss /= len(trainloader.dataset)
        train_acc = 100. * train_correct / train_total

        # Validate the model
        model.eval()
        for batch_idx, (data, target) in enumerate(validloader):
            output = model(data)
            loss = criterion(output, target)

            valid_loss += loss.item() * data.size(0)
            _, pred = torch.max(output, 1)
            valid_correct += torch.sum(pred == target.data)
            valid_total += data.size(0)

        valid_loss /= len(validloader.dataset)
        valid_acc = 100. * valid_correct / valid_total

        # Print training/validation statistics
        print('Epoch: {} \n\tTrain Loss: {:.6f} \tTrain Acc: {:.6f} \n\tValid Loss: {:.6f} \tValid Acc: {:.6f}'.format(
            epoch, train_loss, train_acc, valid_loss, valid_acc))

        # Save the student model if validation loss has decreased
        if is_not_teacher and save_path:
            if valid_loss < valid_loss_min:
                torch.save(model.state_dict(), save_path)
                print('\tValidation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                    valid_loss_min, valid_loss))
                valid_loss_min = valid_loss

    return model


In [ ]:
model = SimpleCNN()
model.to(device)

#define hyperparameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr=0.001)
epochs = 50

In [ ]:
teacher_models = []
i = 1
for trainloader, validloader in zip(trainloaders, validloaders):
  print(" Training Teacher {}".format(i))
  teacher_model = train(epochs, trainloader, validloader, model, optimizer, criterion, True)
  teacher_models.append(teacher_model)
  i+=1
  print("="*40)

 Training Teacher 1
Epoch: 1 
	Train Loss: 25.724381 	Train Acc: 60.294117 
	Valid Loss: 59.330235 	Valid Acc: 52.941177
Epoch: 2 
	Train Loss: 35.746885 	Train Acc: 58.823528 
	Valid Loss: 63.485767 	Valid Acc: 47.058823
Epoch: 3 
	Train Loss: 40.305450 	Train Acc: 57.352940 
	Valid Loss: 55.973114 	Valid Acc: 52.941177
Epoch: 4 
	Train Loss: 39.604138 	Train Acc: 57.352940 
	Valid Loss: 20.732544 	Valid Acc: 58.823528
Epoch: 5 
	Train Loss: 29.253744 	Train Acc: 72.058823 
	Valid Loss: 50.966621 	Valid Acc: 47.058823
Epoch: 6 
	Train Loss: 32.247805 	Train Acc: 63.235294 
	Valid Loss: 28.008081 	Valid Acc: 52.941177
Epoch: 7 
	Train Loss: 21.273771 	Train Acc: 58.823528 
	Valid Loss: 30.709103 	Valid Acc: 52.941177
Epoch: 8 
	Train Loss: 25.647427 	Train Acc: 54.411766 
	Valid Loss: 5.881332 	Valid Acc: 70.588234
Epoch: 9 
	Train Loss: 11.331611 	Train Acc: 55.882355 
	Valid Loss: 15.839268 	Valid Acc: 47.058823
Epoch: 10 
	Train Loss: 9.626131 	Train Acc: 58.823528 
	Valid Loss: 13.

#Getting private student labels

In [ ]:
def student_train_labels(teacher_models, dataloader):
    student_labels = []

    # Get label from each teacher
    for model in teacher_models:
        student_label = []
        for images, _ in dataloader:
            with torch.no_grad():
                # Ensure data is on CPU
                images = images.cpu()
                outputs = model(images)
                preds = torch.argmax(torch.exp(outputs), dim=1)
            student_label.append(preds.tolist())

        # Add all teacher predictions to student_labels
        student_label = sum(student_label, [])
        student_labels.append(student_label)
    return student_labels

# Convert predictions to numpy array and transpose
predicted_labels = student_train_labels(teacher_models, student_train_loader)
predicted_labels = np.array([np.array(p) for p in predicted_labels]).transpose(1, 0)


In [ ]:
# Print the shape of predicted labels and labels of the 3rd image scan
print(predicted_labels.shape)

print(predicted_labels[1])


(163, 5)
[0 0 1 1 1]


#Adding Laplacian noise

The shape of our predicted label is (163, 5), which means there are 163 training examples in our hospital data and each teacher has generated 5 labels for each image. The predicted labels for the 3rd CT image is [0 1 1 0 0]. This means that 3 teacher models agree that the image is negative for COVID-19 while the 2 models disagree. In this case, we select the label with the most votes and conclude that this particular image is negative for COVID-19.

However, there may be cases when the label is not obvious. For example, consider a case with 6 teacher models where 3 of them said the result is positive while the other 3 said it was negative. In that case, we would not be able to reach a consensus and would risk misclassifying the image. This problem can be solved by adding carefully calculated random Laplacian noise to all labels provided by each teacher. Then, get the label with the maximum number of votes from all teachers. The result will be one label for each CT Scan which is a generalized classification of all teacher’s data and is differentially private.

This is called global differential privacy as we are adding noise only after the models have been trained. We choose to add Laplacian noise because it is guaranteed to not leak information of more than epsilon and it is popularly used.

We define an add_noise() method, which takes as input the predicted labels and a value for epsilon (ε). We can control the amount of noise we add using epsilon.

In [ ]:
  def add_noise(predicted_labels, epsilon=0.1):
    noisy_labels = []
    for preds in predicted_labels:

      # get labels with max votes
      label_counts = np.bincount(preds, minlength=2)

      # add laplacian noise to label
      epsilon = epsilon
      beta = 1/epsilon
      for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

      # after adding noise we get labels with max counts
      new_label = np.argmax(label_counts)
      noisy_labels.append(new_label)

    #return noisy_labels
    return np.array(noisy_labels)

  labels_with_noise = add_noise(predicted_labels, epsilon=0.1)
  print(labels_with_noise)
  print(labels_with_noise.shape)

[0 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1
 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 1 1 0 1 0 0 1 0 0 0 0 1 1 1 0
 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0
 1 1 1 0 1 0 0 0 0 1 0 1 1 1 0]
(163,)


In [ ]:
import csv
def write_csv(data):
    with open('labels.csv', 'a') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(data)

write_csv(labels_with_noise)

#Training student data

In [ ]:
def new_student_data_loader(dataloader, noisy_labels, batch_size=32):
  image_list = []
  for image,_ in dataloader:
    image_list.append(image)

  data = np.vstack(image_list)
  new_dataset = list(zip(data, noisy_labels))
  new_dataloader = DataLoader(new_dataset, batch_size, shuffle=False)

  return new_dataloader

labeled_student_trainloader = new_student_data_loader(student_train_loader, labels_with_noise)
len(labeled_student_trainloader),len(student_valid_loader)

(6, 2)

Now the noisy labels generated will be replace with the original labels
In the real life scenario we don't have the original labels

In [ ]:
student_model = train(epochs, labeled_student_trainloader, student_valid_loader, model, optimizer, criterion, save_path='./student.pth.tar', is_not_teacher=True)


Epoch: 1 
	Train Loss: 0.461029 	Train Acc: 80.981598 
	Valid Loss: 0.946850 	Valid Acc: 55.000000
	Validation loss decreased (inf --> 0.946850).  Saving model ...
Epoch: 2 
	Train Loss: 0.294016 	Train Acc: 89.570549 
	Valid Loss: 0.928444 	Valid Acc: 50.000000
	Validation loss decreased (0.946850 --> 0.928444).  Saving model ...
Epoch: 3 
	Train Loss: 0.222762 	Train Acc: 94.478531 
	Valid Loss: 1.299908 	Valid Acc: 47.500000
Epoch: 4 
	Train Loss: 0.167377 	Train Acc: 97.546013 
	Valid Loss: 0.926820 	Valid Acc: 65.000000
	Validation loss decreased (0.928444 --> 0.926820).  Saving model ...
Epoch: 5 
	Train Loss: 0.138463 	Train Acc: 96.932518 
	Valid Loss: 0.791197 	Valid Acc: 55.000000
	Validation loss decreased (0.926820 --> 0.791197).  Saving model ...
Epoch: 6 
	Train Loss: 0.118027 	Train Acc: 97.546013 
	Valid Loss: 1.404068 	Valid Acc: 52.500000
Epoch: 7 
	Train Loss: 0.100912 	Train Acc: 98.159508 
	Valid Loss: 1.118524 	Valid Acc: 50.000000
Epoch: 8 
	Train Loss: 0.087968 

For comparison we train a normal model without using any privacy elements added

In [ ]:
normal_model = train(epochs, student_train_loader, student_valid_loader, model, optimizer, criterion, save_path='./normal.pth.tar', is_not_teacher=True)


Epoch: 1 
	Train Loss: 0.392255 	Train Acc: 80.368095 
	Valid Loss: 0.415316 	Valid Acc: 80.000000
	Validation loss decreased (inf --> 0.415316).  Saving model ...
Epoch: 2 
	Train Loss: 0.315365 	Train Acc: 86.503067 
	Valid Loss: 0.793853 	Valid Acc: 80.000000
Epoch: 3 
	Train Loss: 0.487817 	Train Acc: 78.527611 
	Valid Loss: 0.743210 	Valid Acc: 67.500000
Epoch: 4 
	Train Loss: 0.371402 	Train Acc: 81.595093 
	Valid Loss: 0.521300 	Valid Acc: 85.000000
Epoch: 5 
	Train Loss: 0.378357 	Train Acc: 84.049080 
	Valid Loss: 0.807427 	Valid Acc: 70.000000
Epoch: 6 
	Train Loss: 0.450304 	Train Acc: 80.981598 
	Valid Loss: 0.537664 	Valid Acc: 77.500000
Epoch: 7 
	Train Loss: 0.358463 	Train Acc: 84.049080 
	Valid Loss: 0.680334 	Valid Acc: 72.500000
Epoch: 8 
	Train Loss: 0.403340 	Train Acc: 82.208588 
	Valid Loss: 0.830452 	Valid Acc: 67.500000
Epoch: 9 
	Train Loss: 0.396744 	Train Acc: 80.981598 
	Valid Loss: 0.708359 	Valid Acc: 72.500000
Epoch: 10 
	Train Loss: 0.459762 	Train Acc:

#Comparing the student model and the normal model

Here both the models that we saved will be tested with the

In [ ]:
# Create a dataloader for the test Dataset
batch_size=16
print(len(validset))
dataloader = DataLoader(validset, batch_size=batchsize, shuffle=False)

118


In [ ]:
def test(dataloader, model, criterion):
    # Monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    with torch.no_grad():  # Ensure no gradients are computed during inference
        for batch_idx, (data, target) in enumerate(dataloader):
            # Forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)

            # Calculate the loss
            loss = criterion(output, target)

            # Update average test loss
            test_loss += loss.item() * data.size(0)

            # Convert output probabilities to predicted class
            _, pred = torch.max(output, 1)

            # Compare predictions to true label
            correct += torch.sum(pred == target).item()
            total += data.size(0)

    # Calculate average test loss and accuracy
    test_loss /= len(dataloader.dataset)
    accuracy = 100. * correct / total

    print('\tTest Loss: {:.6f}'.format(test_loss))
    print('\tTest Accuracy: {:.2f}% ({}/{})'.format(accuracy, correct, total))

# Call test function for the student model
print("Student Model")
test(dataloader, student_model, criterion)

print("\n=======================\nNormal Model")
# Call test function for the normal model
test(dataloader, normal_model, criterion)


Student Model
	Test Loss: 0.870982
	Test Accuracy: 64.41% (76.0/118.0)

Normal Model
	Test Loss: 1.023270
	Test Accuracy: 59.32% (70.0/118.0)


The values (61.0/118.0) and (70.0/118.0) represent the fraction of correctly classified samples over the total number of samples in the test dataset.


Our newly created label provided much better accuracy than the regular model